In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [3]:
def set_seed(seed: int) -> None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed: int = 42
set_seed(seed)

In [4]:
device:str = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device.type

'cuda'

In [5]:
def text_generator(text:str, seq_length:int, vocab_to_idx:dict):
    text_length = len(text)
    while True:
        for i in range(0, text_length - seq_length, seq_length):
            input_text = text[i:i + seq_length]
            target_text = text[i + 1:i + seq_length + 1]

            input_ids = torch.tensor([vocab_to_idx[char] for char in input_text], dtype=torch.long)
            target_ids = torch.tensor([vocab_to_idx[char] for char in target_text], dtype=torch.long)

            yield input_ids, target_ids

In [6]:
import requests


URL = "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"
response = requests.get(URL)

if response.status_code != 200:
    exit()

text = response.text

text[:60]



'First Citizen:\nBefore we proceed any further, hear me speak.'

In [7]:
vocab:set[str] = sorted(set(text))
vocab_to_idx:dict[str, int] = {char: idx for idx, char in enumerate(vocab)}
idx_to_vocab:dict = {idx: char for char, idx in vocab_to_idx.items()}

seq_length:int = 300
generator = text_generator(text, seq_length, vocab_to_idx)
BATCH_SIZE = 64
def collate_fn(batch:list):
    inputs, targets = zip(*batch)
    inputs = torch.stack(inputs)
    targets = torch.stack(targets)
    return inputs, targets

dataloader = DataLoader(generator, batch_size=BATCH_SIZE, collate_fn=collate_fn, drop_last=True)


In [8]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size:int, embedding_dim:int, rnn_units:int):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, rnn_units, batch_first=True)
        self.fc = nn.Linear(rnn_units, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden

In [9]:
vocab_size:int = len(vocab)
embedding_dim:int = 256
rnn_units:int = 1024

model = RNNModel(vocab_size, embedding_dim, rnn_units).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
def train_model(model, generator, criterion, optimizer, epochs:int, batch_size:int) -> None:
    model.train()
    for epoch in range(epochs):
        hidden = None
        step = 0
        while step < (len(text) // (seq_length * batch_size)):
            step += 1
            batch = [next(generator) for _ in range(batch_size)]
            inputs, targets = zip(*batch)
            inputs = torch.stack(inputs).to(device)
            targets = torch.stack(targets).to(device)

            optimizer.zero_grad()

            outputs, hidden = model(inputs, hidden)
            hidden = hidden.detach()

            loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
            loss.backward()
            optimizer.step()

            print(f"Epoch {epoch + 1}/{epochs}, Step {step}, Loss: {loss.item():.4f}", end='\r')
        print()

In [11]:
EPOCHS:int = 30
train_model(model, generator, criterion, optimizer, EPOCHS, BATCH_SIZE)


Epoch 1/30, Step 58, Loss: 2.1285
Epoch 2/30, Step 58, Loss: 1.9055
Epoch 3/30, Step 58, Loss: 1.7853
Epoch 4/30, Step 58, Loss: 1.6942
Epoch 5/30, Step 58, Loss: 1.6193
Epoch 6/30, Step 58, Loss: 1.5565
Epoch 7/30, Step 58, Loss: 1.5054
Epoch 8/30, Step 58, Loss: 1.4666
Epoch 9/30, Step 58, Loss: 1.4426
Epoch 10/30, Step 58, Loss: 1.4055
Epoch 11/30, Step 58, Loss: 1.3664
Epoch 12/30, Step 58, Loss: 1.3351
Epoch 13/30, Step 58, Loss: 1.2896
Epoch 14/30, Step 58, Loss: 1.2616
Epoch 15/30, Step 58, Loss: 1.2114
Epoch 16/30, Step 58, Loss: 1.1794
Epoch 17/30, Step 58, Loss: 1.1371
Epoch 18/30, Step 58, Loss: 1.1038
Epoch 19/30, Step 58, Loss: 1.0688
Epoch 20/30, Step 58, Loss: 1.0486
Epoch 21/30, Step 58, Loss: 1.0260
Epoch 22/30, Step 58, Loss: 0.9971
Epoch 23/30, Step 58, Loss: 0.9814
Epoch 24/30, Step 58, Loss: 0.9575
Epoch 25/30, Step 58, Loss: 0.9292
Epoch 26/30, Step 58, Loss: 0.9093
Epoch 27/30, Step 58, Loss: 0.8927
Epoch 28/30, Step 58, Loss: 0.8974
Epoch 29/30, Step 58, Loss: 0

In [12]:
import torch
save_path = "model.pth"
torch.save(model.state_dict(), save_path)

In [13]:
def generate_text(model, start_text:str, length:int, idx_to_vocab:dict[int, str], vocab_to_idx:dict[str, int], temperature:float=1.0) -> str:
    model.eval()
    input_ids = torch.tensor([vocab_to_idx[char] for char in start_text], dtype=torch.long).unsqueeze(0).to(device)
    hidden = None

    generated_text = start_text
    print(generated_text, end="")
    for _ in range(length):
        with torch.no_grad():
            outputs, hidden = model(input_ids, hidden)
            logits = outputs[:, -1, :] / temperature
            probs = torch.nn.functional.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1).item()

        next_char = idx_to_vocab[next_id]
        print(next_char, end="")

        generated_text += next_char

        input_ids = torch.tensor([[next_id]], dtype=torch.long).to(device)

    return generated_text

In [14]:
start_text:str = "ROMEO:"
generated_text:str = generate_text(model, start_text, 1000, idx_to_vocab, vocab_to_idx, 1.0)


ROMEO:

Ghost of Frand, I would make me such a preck.

MONTAGUE:
You must pawns by thee, to this gettle Gaunt;
For devil's called consideres
Thoughts are not, sir:
I'll tell her my hand.

PAULINA:
I hope my senseless; but is no meet.

LUCENTIO:
Go to, go to:
Gold Bohen!--

DUKE VINCENTIO:
Mend an edact: a ring to seem thee.

GLOUCESTER:

KING EDWARD IV:
Lest he had been aside,
And the summon'd of the vews front; I shall
It steal my assurance made me burable:
I must have show'd to-morrow, giddy, fellow;
So falsed as youbsteed, old favour
Inspurn a sudden will take her hence:
And yet makes for me.

GLOUCESTER:

ISABELLA:
What, ho! what will betroked?
But in pluck him and left me sich a good
which sire; contains God tell her maid.

FADY CAPULET:
Verona, some son of liberty.

LUCIO:

ISABELLA:
Who is't that were I appay?

LUCIO:
I pray.

LUCIO:
Have you no none here? mether.

Provost:
Do stop there, fool, if I tell me
Af our prisoner? Camillo!

Aft think me: there is,
Unless abroad unconsi